In [1]:
from app import create_app, db
from app.instrument import Instrument
from app.analysis import Quote
mappings= {'BRK.B': 'BRK-B'}
with create_app().app_context():
    instruments = {b.symbol if b.symbol not in mappings else mappings[b.symbol]:
                   b for b in Instrument.find_stocks(200)}
    quote = Quote([*instruments.keys()], 120)
    quote.update_boosts(21, instruments)
    db.session.commit()
f'from {quote.start.date()} to {quote.end.date()} - {len(quote.data.columns)}'

'from 2019-02-25 to 2019-06-21 - 200'

In [2]:
stat = quote.statistics(21)
stat = stat[(stat['drawdown'] < 8) & (stat['21-mean'] > 0.8)]
stat

,21-len,21-mean,21-std,21-shrp,drawdown
Symbols,,,,,
KO,62,3.925656,1.653503,2.273348,3.383609
MCD,62,3.517417,1.484106,2.257757,2.999196
MA,62,4.373865,1.986382,2.118020,5.698715
WM,62,3.976322,2.068442,1.841800,3.316666
SHOP,62,15.249269,8.838574,1.706452,6.171438
SBUX,62,4.505872,2.857253,1.518663,4.600183
V,62,3.602156,2.406417,1.427637,4.317938
PYPL,62,4.741584,3.352832,1.364493,7.332929
ENPH,62,24.747480,18.234668,1.348026,6.170212


In [5]:
quote.setup_mask(stat.index)
quote.least_correlated_portfolio(21, 3, cr=3, dr=4)

['KO', 'MCD', 'MA'] 6.3924378119809635 1.1254486802220485 2.4163041921731514 -6.649124997377788
['KO', 'MCD', 'WM'] 4.039901227969651 0.8844094418585375 1.9398944191454475 -6.372904774187752
['KO', 'MCD', 'ENPH'] 2.8750411750443856 -0.4294139595428239 2.5106035628583676 -5.879131197760612
['MCD', 'WM', 'ENPH'] 2.720754350450143 -0.6068409234841616 2.497214999406526 -5.447582876723477


['MCD', 'WM', 'ENPH']

In [6]:
quote.setup_mask(['MCD', 'WM', 'ENPH'])
quote.find_optimal_ratio(21, 4)

     fun: 1.0962636767881013
     jac: array([2.19325539, 2.19284588, 2.17431054])
 message: 'Optimization terminated successfully.'
    nfev: 20
     nit: 4
    njev: 4
  status: 0
 success: True
       x: array([0.65063477, 0.31781642, 0.03154881])


/Users/chaoyi/.local/share/virtualenvs/portfolio_analysis-bdMx2S3t/lib/python3.7/site-packages/scipy/optimize/minpack.py:162: RuntimeWarning: The iteration is not making good progress, as measured by the 
  improvement from the last ten iterations.
  warnings.warn(msg, RuntimeWarning)


({'MCD': 0.6506347736047629,
  'WM': 0.31781641932672217,
  'ENPH': 0.03154880706851493},
 4.333,
 1.047)

In [ ]:
%pylab inline
quote.graph(21, {'MCD': 0.65, 'WM': 0.32, 'ENPH': 0.03})